## Proyecto 3 - SDS
### Sara Zavala

In [1]:
##Ataque de extracción 

In [2]:
import pprint
import numpy as np
import pandas as pd
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn import *
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [3]:
path = ("malimg_paper_dataset_imgs")

In [4]:
path_images = ImageDataGenerator().flow_from_directory(directory=path, target_size=(128,128), batch_size=10000)

Found 9339 images belonging to 25 classes.


Ataque de evasión 
1. Se debe mostrar un entrenamiento y predicción del modelo original 
2. Utilizando el ataque de evasión Fast Gradient Method genere observaciones falsas 
3. Muestre  la  cantidad  de  observaciones  generadas  que  fueron  clasificadas  correcta  e 
incorrectamente 
4. Explique con detalle los resultados obtenidos. ¿Cómo podría proteger su modelo ante este 
tipo de ataques? 
5. Realce las modificaciones que considere pertinentes para hacer su modelo robusto contra este  tipo  de  ataques  (puede  modificar  el  modelo  original,  así  como  el  dataset  de entrenamiento). Repita los pasos 1 al 3 con el nuevo modelo robusto y demuestre que es menos susceptible a equivocarse con observaciones generadas 

In [5]:
path_images.class_indices

{'Adialer.C': 0,
 'Agent.FYI': 1,
 'Allaple.A': 2,
 'Allaple.L': 3,
 'Alueron.gen!J': 4,
 'Autorun.K': 5,
 'C2LOP.P': 6,
 'C2LOP.gen!g': 7,
 'Dialplatform.B': 8,
 'Dontovo.A': 9,
 'Fakerean': 10,
 'Instantaccess': 11,
 'Lolyda.AA1': 12,
 'Lolyda.AA2': 13,
 'Lolyda.AA3': 14,
 'Lolyda.AT': 15,
 'Malex.gen!J': 16,
 'Obfuscator.AD': 17,
 'Rbot!gen': 18,
 'Skintrim.N': 19,
 'Swizzor.gen!E': 20,
 'Swizzor.gen!I': 21,
 'VB.AT': 22,
 'Wintrim.BX': 23,
 'Yuner.A': 24}

In [6]:
data = path_images.class_indices
data2 = list(data.values())
print(len(data2)-1)

24


In [7]:
# x, y = next(batches)
images, marks = next(path_images)

In [9]:
images.shape

(9339, 128, 128, 3)

In [10]:
images.size

459030528

In [11]:
marks.size

233475

In [12]:
print('Images size --> ', images.itemsize)
print('Marks --> ',marks.itemsize)

Images size -->  4
Marks -->  4


In [14]:
images

array([[[[201., 201., 201.],
         [ 46.,  46.,  46.],
         [114., 114., 114.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[210., 210., 210.],
         [245., 245., 245.],
         [252., 252., 252.],
         ...,
         [185., 185., 185.],
         [ 52.,  52.,  52.],
         [254., 254., 254.]],

        ...,

        [[235., 235., 235.],
         [ 54.,  54.,  54.],
         [185., 185., 185.],
         ...,
         [184., 184., 184.],
         [203., 203., 203.],
         [  7.,   7.,   7.]],

        [[ 54.,  54.,  54.],
         [165., 165., 165.],
         [  6.,   6.,   6.],
         ...,
         [ 19.,  19.,  19.],
         [115., 115., 115.],
         [ 51.,  51.,  51.]],

        [[142., 142., 142.],
       

In [13]:
marks

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
## Entrenamiento y prediccion

In [15]:
X_train, X_test0, y_train, y_test0 = train_test_split(images, marks, test_size = 0.2)